In [53]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sae_lens import SAE, HookedSAETransformer

In [54]:

model = HookedSAETransformer.from_pretrained("gpt2-small", device=device)
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb-feature-splitting",  # <- Release name
    sae_id="blocks.8.hook_resid_pre_768",  # <- SAE id (not always a hook point!)
    device=device,
)

Loaded pretrained model gpt2-small into HookedTransformer


In [55]:
# Input sentence
input_sentence = "Who are you? I am"
inputs = model.tokenizer(input_sentence, return_tensors="pt").to(device)

# Get model outputs
with torch.no_grad():
    logits, activations = model.run_with_cache(input_sentence)

print(sae.encode_standard(activations['blocks.8.hook_resid_pre']).shape)

torch.Size([1, 7, 768])


In [25]:
sae.decode(sae.encode_standard(activations['blocks.8.hook_resid_pre'])).shape

torch.Size([1, 7, 768])

In [88]:
def steering_features(value, hook):
    # BERT 生成的768vector*activation vector
    
    #encoded_activation = sae.encode(value)
    #steered_vector = steering_vector*encoded_activation 
    #value = sae.decode(steered_vector)
    return value
    

fwd_hooks=[(
        'blocks.8.hook_resid_pre', 
        steering_features
        )]
prompt = "I am a person who"

with model.hooks(fwd_hooks=fwd_hooks):
    steered_text = model.generate(
        prompt,
        max_new_tokens=10,
        temperature=0.2,
        top_p=0.9,
        stop_at_eos = True,
        
    )

print(steered_text)

AttributeError: 'FASG_Model' object has no attribute 'hooks'

In [45]:
## pseudo code

# Step 1: Use BERT to predict the 768 vector -> steering vector

# Our own model
class BertWithDense(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", output_dim=768):
        pass
    def forward(self, input_ids, attention_mask):
        pass

bert_model = BertWithDense()
#bert_model(tokenizer(x)) <--768 vector

# Step 2: Generate Text with LLM
from sae_lens import SAE, HookedSAETransformer

# model
model = HookedSAETransformer.from_pretrained("gpt2-small", device=device)

# sae
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb-feature-splitting",  # <- Release name
    sae_id="blocks.8.hook_resid_pre_768",  # <- SAE id (not always a hook point!)
    device=device,
)

def steering_features(value, hook):
    # BERT 生成的768vector*activation vector
    
    #encoded_activation = sae.encode(value)
    #steered_vector = steering_vector*encoded_activation 
    #value = sae.decode(steered_vector)
    return value 

fwd_hooks=[(
        'blocks.8.hook_resid_pre', 
        steering_features
        )]


prompt = x[:10]

with model.hooks(fwd_hooks=fwd_hooks):
    steered_text = model.generate(
        prompt,
        max_new_tokens=10,
        temperature=0.2,
        top_p=0.9,
        stop_at_eos = True,
        
    )

# Step 3 -> calculate loss and gradient descent
# loss =  cosine_similarity(x,steered_text)
# with bert_model

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sae_lens import SAE, HookedSAETransformer
import torch
from torch import nn
from transformers import BertModel, BertTokenizer


class FASG_Model(nn.Module):
    def __init__(self):
        super(FASG_Model, self).__init__()

        self.device = "cuda:1"
        self.bertTokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bertModel = BertModel.from_pretrained("bert-base-uncased").to(self.device)
        self.linear = torch.nn.Linear(768, 768).to(self.device)
        self.llm = HookedSAETransformer.from_pretrained("gpt2-small", device=self.device)
        self.sae, cfg_dict, sparsity = SAE.from_pretrained(
            release="gpt2-small-res-jb-feature-splitting",  
            sae_id="blocks.8.hook_resid_pre_768", 
            device=self.device,
        )
        

    def forward(self, text):
        encoded_input = tokenizer(text, return_tensors='pt',padding=True, truncation=True)
        steering_vector = self.bertModel(**encoded_input).pooler_output

        def steering_features(value, hook):
            encoded_activation = self.sae.encode(value)
            print(encoded_activation.shape)
            print(steering_vector.shape)
            steered_vector = steering_vector*encoded_activation 
            decoded_vector = sae.decode(steered_vector)
            return decoded_vector
    
        fwd_hooks=[(
            'blocks.8.hook_resid_pre', 
            steering_features
        )]

        prompt = " ".join(text.split(" ")[:7])
        
        with self.llm.hooks(fwd_hooks=fwd_hooks):
            steered_tokens = self.llm.generate(
            prompt,
            max_new_tokens=64,
            temperature=0.2,
            top_p=0.9,
            stop_at_eos = True,
            return_type = "tensor"
        )
        return steered_tokens

In [5]:
sample_text = "this is a sample text"
model = FASG_Model()
result= model(sample_text)

Loaded pretrained model gpt2-small into HookedTransformer


/home/jupyter-qj351/.conda/envs/small-data/lib/python3.11/site-packages/sae_lens/sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


NameError: name 'tokenizer' is not defined

In [118]:
torch.cuda.devices[1]

AttributeError: module 'torch.cuda' has no attribute 'devices'